In [86]:
%pip install numpy opencv-python
%pip install --user mediapipe

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import mediapipe as media
import numpy as np 

mp_drawing_styles = media.solutions.drawing_styles
mp_drawing = media.solutions.drawing_utils
mp_pose = media.solutions.pose

In [2]:
def preprocess_frame(frame):
    H, W = frame.shape[0], frame.shape[1]
    resizeH = 600
    resizeW = int((W / H) * resizeH)
    
    input_image = cv2.resize(frame, (resizeW, resizeH))
    #input_image = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)    #makes you blue!
    
    return input_image

In [7]:
########    PROCESS SAVED VIDEO DATA    ########

#open video file
vidURL = './data/training/reject.mp4'
cap = cv2.VideoCapture(vidURL)

with mp_pose.Pose(min_detection_confidence=0.25, min_tracking_confidence=0.25) as pose:
    while cap.isOpened():
        #read and preprocess frame
        ret, frame = cap.read()
        if not ret: 
            break
    
        #show feed!
        frame = preprocess_frame(frame) 

        results = pose.process(frame)

        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec = mp_drawing_styles.get_default_pose_landmarks_style())
    
        cv2.imshow('Video feed', frame)

#clean up and close program
cap.release()
cv2.destroyAllWindows()

In [3]:
########    PROCESS LIVE WEBCAM DATA    ########

#open capture
cap = cv2.VideoCapture(0)
pose_landmarks = []

with mp_pose.Pose(min_detection_confidence=0.25, min_tracking_confidence=0.25) as pose:
    while cap.isOpened():
        #read and preprocess frame
        ret, frame = cap.read()
        if not ret: 
            break
            
        frame = preprocess_frame(frame)

        results = pose.process(frame)

        #draw pose annotation
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec = mp_drawing_styles.get_default_pose_landmarks_style())
        
        #show feed!
        cv2.imshow('Live feed', frame)
    
        #listen for 'Q' key
        if cv2.waitKey(1) == ord('q'):
            pose_landmarks = results.pose_landmarks
            break

#clean up and close program
cap.release()
cv2.destroyAllWindows()

c:\Users\benlc\hiphopml\env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [9]:
########    match pose landmarks to coordinates     ########

pose_indices = ['nose', 
                'left_eye_inner', 'left_eye', 'left_eye_outer', 
                'right_eye_inner', 'right_eye', 'right_eye_outer',
                'left_ear', 'right_ear',
                'mouth_left', 'mouth_right',
                'left_shoulder', 'right_shoulder',
                'left_elbow', 'right_elbow',
                'left_wrist', 'right_wrist',
                'left_pinky', 'right_pinky',
                'left_index', 'right_index',
                'left_thumb', 'right_thumb',
                'left_hip', 'right_hip',
                'left_knee', 'right_knee',
                'left_ankle', 'right_ankle',
                'left_heel', 'right_heel',
                'left_foot_index', 'right_foot_index']

def setNewCoords(landmark):
    new_coords = [{
                'x': point.x,
                'y': point.y,
                'z': point.z,
                'vis': point.visibility
                }   
                for point in landmark]

    keypoints = {}
    for i in range(len(pose_indices)):
        keypoints[pose_indices[i]] = new_coords[i]

    return keypoints

keypoints = setNewCoords(pose_landmarks.landmark)
for key in keypoints.keys():
    if keypoints[key]['vis'] > 0.5:
        print(key, keypoints[key])

nose {'x': 0.621558427810669, 'y': 0.08921728283166885, 'z': -0.5160354375839233, 'vis': 0.9999528527259827}
left_eye_inner {'x': 0.6334375739097595, 'y': 0.07345728576183319, 'z': -0.484919011592865, 'vis': 0.9999215602874756}
left_eye {'x': 0.6378164291381836, 'y': 0.07472570985555649, 'z': -0.4850243031978607, 'vis': 0.9999128580093384}
left_eye_outer {'x': 0.6419649720191956, 'y': 0.07605085521936417, 'z': -0.4851114749908447, 'vis': 0.999923586845398}
right_eye_inner {'x': 0.6195680499076843, 'y': 0.0685989037156105, 'z': -0.4895278811454773, 'vis': 0.9999054074287415}
right_eye {'x': 0.6142794489860535, 'y': 0.06626370549201965, 'z': -0.4896296560764313, 'vis': 0.9998775124549866}
right_eye_outer {'x': 0.6092092990875244, 'y': 0.06397171318531036, 'z': -0.48960861563682556, 'vis': 0.9998661875724792}
left_ear {'x': 0.645785391330719, 'y': 0.0829315185546875, 'z': -0.2932945787906647, 'vis': 0.9998888373374939}
right_ear {'x': 0.6009838581085205, 'y': 0.06389220803976059, 'z': -0.

In [13]:
########    calculate parameter values     ########

def calcParamValues(keypoints):
    #hip midpoint
    hip_midpoint = (keypoints['right_hip']['x'] - keypoints['left_hip']['x']) / 2

    #calculate cartesian distance between two points
    def calculateDistance(x, y):
        return np.sqrt(x ** 2 + y ** 2)

    #calculate angle created by two cartesian vectors
    #   negative angles : bend outward, i.e. gotta piss
    #   positive angles : bend inward, i.e. sumo squat
    def calculateAngle(A, B, C = None):
        direction = 1
        if not C:
            C = {'x': 1, 'y': 0}
        elif abs(B['x'] - hip_midpoint) < abs(C['x'] - hip_midpoint):
            direction = -1

        vecBA, vecBC = [A['x'] - B['x'], A['y'] - B['y']], [C['x'] - B['x'], C['y'] - B['y']]
        dot_product = vecBA[0] * vecBC[0] + vecBA[1] * vecBC[1]
        magBA, magBC = calculateDistance(vecBA[0], vecBA[1]), calculateDistance(vecBC[0], vecBC[1])
        return np.arccos(dot_product / (magBA * magBC)) * direction

    #calculate body facing
    # -pi : facing to the right
    #  0  : facing the camera
    #  pi : facing to the left
    shoulder_depthdiff = keypoints['left_shoulder']['z'] - keypoints['right_shoulder']['z']
    shoulder_widthdiff = keypoints['left_shoulder']['x'] - keypoints['right_shoulder']['x']
    body_angle = np.arctan(shoulder_depthdiff / shoulder_widthdiff)

    #calculate arm angles
    leftarm_angle = calculateAngle(keypoints['left_shoulder'], keypoints['left_elbow'], keypoints['left_wrist']) 
    rightarm_angle = calculateAngle(keypoints['right_shoulder'], keypoints['right_elbow'], keypoints['right_wrist']) 
    leftleg_angle = calculateAngle(keypoints['left_hip'], keypoints['left_knee'], keypoints['left_ankle'])
    rightleg_angle = calculateAngle(keypoints['right_hip'], keypoints['right_knee'], keypoints['right_ankle'])

    #calculate shoulder dimensions relative to hips
    shoulder_angle = calculateAngle(keypoints['left_shoulder'], keypoints['right_shoulder'])
    hip_angle = calculateAngle(keypoints['left_hip'], keypoints['right_hip'])
    shoulder_relativeangle = shoulder_angle - hip_angle

    shoulder_midpoint = (keypoints['right_shoulder']['x'] - keypoints['left_shoulder']['x']) / 2
    shoulder_relativemidpoint = abs(shoulder_midpoint - hip_midpoint)

    #calculate foot dimensions relative to each other
    foot_depthdiff = keypoints['left_ankle']['z'] - keypoints['left_ankle']['z']
    foot_heightdiff = keypoints['left_ankle']['y'] - keypoints['right_ankle']['y']

    #calculate angle of the foot
    # using knee-heel-toe angle to describe foot direction
    leftfoot_angle = calculateAngle(keypoints['left_knee'], keypoints['left_heel'], keypoints['left_foot_index'])
    rightfoot_angle = calculateAngle(keypoints['right_knee'], keypoints['right_heel'], keypoints['right_foot_index'])

    profile = {'body-ang': body_angle, 
               'LA-ang': leftarm_angle, 'RA-ang': rightarm_angle, 
               'LL-ang': leftleg_angle, 'RL-ang': rightleg_angle,
            'SR-ang': shoulder_relativeangle, 'SR-mp': shoulder_relativemidpoint,
            'F-depth': foot_depthdiff, 'F-height': foot_heightdiff,
            'LF-ang': leftfoot_angle, 'RF-ang': rightfoot_angle}
    
    return profile

parameterValues = calcParamValues(keypoints)
for key in parameterValues.keys():
    print(key, parameterValues[key])

body-ang 0.2968405428715714
LA-ang 1.9901597463662961
RA-ang 2.040441019913823
LL-ang -2.9303926250519243
RL-ang 3.1332781667736183
SR-ang 0.02653235396175402
SR-mp 0.020781755447387695
F-depth 0.0
F-height 0.015932857990264893
LF-ang -2.3055504388264145
RF-ang -2.682542757841427
